In [2]:
pip install bitstring

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 748.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 3.3 MB/s eta 0:00:00


In [7]:
from bitstring import BitArray

block_size = 32
half_block_size = block_size // 2

CUSTOM_EXPANSION_TABLE = [
    2, 1, 4, 3, 7, 8, 9, 6,10, 5, 12, 11, 14, 13, 16, 15,
    1, 2, 3, 4, 5, 6, 7, 8,9, 10, 11, 12, 13, 14, 15, 16
]

CUSTOM_PERMUTATION_TABLE = [
    2, 1, 4, 3, 7, 8, 9, 6,10, 5, 12, 11, 14, 13, 16, 15,
    20, 18, 17, 19, 24, 22, 21, 23,28, 26, 25, 27, 32, 30, 29, 31
]



def string_to_binary(input_string):
    return ''.join(format(ord(char), '08b') for char in input_string)


def binary_to_string(input_binary):
    return ''.join(chr(int(input_binary[i:i+8], 2)) for i in range(0, len(input_binary), 8))

def permute(block, table):
    # Ensure block is at least as long as the highest index in table
    if len(block) < max(table):
        block = block.ljust(max(table), '0')
    return ''.join(block[i - 1] for i in table)

def split(block):
    half_length = len(block) // 2
    return block[:half_length], block[half_length:]

def combine(left, right):
    return left + right

def expand(block, table):
    return ''.join(block[i - 1] for i in table)

def feistelNetwork(block, keys):
    left, right = split(block)
    for round_key in keys:
        expanded_right = expand(right, CUSTOM_EXPANSION_TABLE)
        mixed = ''.join(str(int(a) ^ int(b)) for a,b in zip(expanded_right , round_key))
        permuted_mixed = permute(mixed , CUSTOM_PERMUTATION_TABLE)
        new_right = ''.join(str(int(a) ^ int(b)) for a,b in zip(left , permuted_mixed))
        left = right
        right = new_right
    return combine(right , left)

def pad_key_to_64_bits(key):
    return key.rjust(64 , '0')

def generate_round_subkeys(key , num_rounds):
    subkey_length = len(key) // num_rounds
    return [key[i * subkey_length:(i +1) * subkey_length] for i in range(num_rounds)]

def encrypt(plaintext, keys):
    return feistelNetwork(plaintext, keys)

def decrypt(ciphertext, keys):
    reversed_keys = keys[::-1]
    return feistelNetwork(ciphertext, reversed_keys)

# Example:
key = '1010101010101010101010101010101010101010101010101010101010101010' # 64-bit key
padded_key = pad_key_to_64_bits(key)
print('Padded Key:' , padded_key)
round_subkeys = generate_round_subkeys(padded_key ,8)
print('Round Subkeys:' , round_subkeys)
plaintext = 'narges'
# Convert plaintext to binary and ensure it's the correct size (block_size bits)
binary_plaintext = string_to_binary(plaintext).ljust(block_size , '0')[:block_size]
encrypted_binary = encrypt(binary_plaintext , round_subkeys)
print('Encrypted Binary:' , encrypted_binary)
decrypted_binary = decrypt(encrypted_binary , round_subkeys)
print('Decrypted Binary:' , decrypted_binary)
# Check if original binary plaintext matches decrypted binary
if binary_plaintext == decrypted_binary:
    print('Success: Original plaintext matches decrypted text')
else:
    print('Error: Original plaintext does not match decrypted text')

decrypted_message = binary_to_string(decrypted_binary)
print('Decrypted Message:', decrypted_message)


Padded Key: 1010101010101010101010101010101010101010101010101010101010101010
Round Subkeys: ['10101010', '10101010', '10101010', '10101010', '10101010', '10101010', '10101010', '10101010']
Encrypted Binary: 01100110011001110100111001100001
Decrypted Binary: 01101110011000010111001001100111
Success: Original plaintext matches decrypted text
Decrypted Message: narg


In [3]:
from bitstring import BitArray

block_size = 32
half_block_size = block_size // 2

CUSTOM_EXPANSION_TABLE = [
    2, 1, 4, 3, 7, 8, 9, 6,10, 5, 12, 11, 14, 13, 16, 15,
    1, 2, 3, 4, 5, 6, 7, 8,9, 10, 11, 12, 13, 14, 15, 16
]

CUSTOM_PERMUTATION_TABLE = [
    2, 1, 4, 3, 7, 8, 9, 6,10, 5, 12, 11, 14, 13, 16, 15,
    20, 18, 17, 19, 24, 22, 21, 23,28, 26, 25, 27, 32, 30, 29, 31
]

def string_to_binary(input_string):
    return ''.join(format(ord(char), '08b') for char in input_string)

def binary_to_string(input_binary):
    return ''.join(chr(int(input_binary[i:i+8], 2)) for i in range(0, len(input_binary), 8))

def permute(block, table):
    # Ensure block is at least as long as the highest index in table
    if len(block) < max(table):
        block = block.ljust(max(table), '0')
    return ''.join(block[i - 1] for i in table)

def split(block):
    half_length = len(block) // 2
    return block[:half_length], block[half_length:]

def shift(bits, n):
    return bits[n:] + bits[:n]

def combine(left, right):
    return left + right

def expand(block, table):
    return ''.join(block[i - 1] for i in table)

def feistelNetwork(block, keys):
    left, right = split(block)
    for round_key in keys:
        expanded_right = expand(right, CUSTOM_EXPANSION_TABLE)
        mixed = ''.join(str(int(a) ^ int(b)) for a,b in zip(expanded_right , round_key))
        permuted_mixed = permute(mixed , CUSTOM_PERMUTATION_TABLE)
        new_right = ''.join(str(int(a) ^ int(b)) for a,b in zip(left , permuted_mixed))
        left = right
        right = new_right
    return combine(right , left)

def pad_key_to_64_bits(key):
    return key.rjust(64 , '0')

def generate_round_subkeys(key , num_rounds):
    subkey_length = len(key) // num_rounds
    return [key[i * subkey_length:(i +1) * subkey_length] for i in range(num_rounds)]
def encrypt(plaintext, keys):
    return feistelNetwork(plaintext, keys)

def decrypt(ciphertext, keys):
    reversed_keys = keys[::-1]
    return feistelNetwork(ciphertext, reversed_keys)
def process_in_blocks(text, block_size, process_function):
    # Pad text to be a multiple of block_size
    padded_text = text.ljust((len(text) + block_size - 1) // block_size * block_size, '0')
    result = ''
    for i in range(0, len(padded_text), block_size):
        block = padded_text[i:i+block_size]
        result += process_function(block)
    return result


# Example:
key = '1010101010101010101010101010101010101010101010101010101010101010' # 64-bit key

padded_key = pad_key_to_64_bits(key)
print('Padded Key:' , padded_key)

round_subkeys = generate_round_subkeys(padded_key ,8)
print('Round Subkeys:' , round_subkeys)


plaintext = 'nargesdb'
# Convert plaintext to binary
binary_plaintext = string_to_binary(plaintext)
# Process binary plaintext in blocks and encrypt
encrypted_binary = process_in_blocks(binary_plaintext, block_size, lambda block: encrypt(block.ljust(block_size, '0')[:block_size], round_subkeys))
print('Encrypted Binary:', encrypted_binary)
# Process encrypted binary in blocks and decrypt
decrypted_binary = process_in_blocks(encrypted_binary, block_size, lambda block: decrypt(block.ljust(block_size, '0')[:block_size], round_subkeys))
print('Decrypted Binary:', decrypted_binary)
# Convert decrypted binary back to string
decrypted_message = binary_to_string(decrypted_binary)
print('Decrypted Message:', decrypted_message)
# Check if original binary plaintext matches decrypted binary
if binary_plaintext == decrypted_binary:
    print('Success: Original plaintext matches decrypted text')
else:
    print('Error: Original plaintext does not match decrypted text')

Padded Key: 1010101010101010101010101010101010101010101010101010101010101010
Round Subkeys: ['10101010', '10101010', '10101010', '10101010', '10101010', '10101010', '10101010', '10101010']
Encrypted Binary: 0110011001100111010011100110000101100101011000100101000101110011
Decrypted Binary: 0110111001100001011100100110011101100101011100110110010001100010
Decrypted Message: nargesdb
Success: Original plaintext matches decrypted text
